In [1]:
!pip install sentence-transformers scikit-learn pandas numpy

In [2]:
import pandas as pd

df = pd.read_csv("transactions.csv")
df.head()


,date,amount,description,category,type,payment_method,installments,monthly_payment
0,2024-01-22,1.00,University tuition payment,Education,expense,cash,0,0.00
1,2023-04-04,1.00,Gas bill,Utilities,expense,debit_card,0,0.00
2,2023-11-19,4.95,Aldi grocery,Food,expense,credit_card,1,4.95
3,2023-08-17,86.88,University tuition payment,Education,expense,debit_card,0,0.00
4,2024-09-23,35.26,Subway sandwich,Food,expense,debit_card,0,0.00


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeddings = model.encode(df["description"].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X = embeddings
y = df["category"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = SVC(kernel="linear", probability=True)
clf.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [6]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

                precision    recall  f1-score   support

     Education       1.00      1.00      1.00        45
 Entertainment       1.00      1.00      1.00       109
          Food       1.00      1.00      1.00       243
        Health       1.00      1.00      1.00        55
        Income       1.00      1.00      1.00       138
      Shopping       1.00      1.00      1.00       134
Transportation       1.00      1.00      1.00       181
     Utilities       1.00      1.00      1.00        95

      accuracy                           1.00      1000
     macro avg       1.00      1.00      1.00      1000
  weighted avg       1.00      1.00      1.00      1000



In [7]:
import joblib

joblib.dump(clf, "category_classifier_sbert.pkl")
model.save("minilm_embedding_model")